Основная часть (20 баллов)


1. Сделайте генератор, который принимает на вход путь к фаста файлу и выдаёт по очереди пары id последовательности (то, что после ">") и последовательности. (5 баллов)
Пример использования на рисунке Example1. Обратите внимание, что функция должна возвращать НЕ список, а генератор! Биопитон и другие библиотеки не используем.


2. Напишите класс, производящий чтение последовательностей с небольшими изменениями. (15 баллов)
Класс должен иметь конструктор хотя бы с одним аргументом - путь к фаста файлу
Объект данного класса должен поддерживать итерацию по нему. Не по атрибутам, а именно по самому объекту.
В процессе итерации класс бесконечно перебирает последовательности в файле. Если файл закончился, то итерация продолжается с его начала.
При возвращении каждой очередной последовательности класс немного изменяет её с заданной вероятностью (способ задачи вероятности придумайте сами). Можно менять часть аминокислот, делать делеции, вставки и т.д.. Функционал для изменения последовательностей выделите в отдельный(е) метод(ы).
Наследоваться запрещается


Дополнительная часть (10 баллов)


Чтобы выполнить эти задания, понадобится найти дополнительную информацию об итераторах.


1. Напишите генератор iter_append(iterable, item), который "добавляет" элемент item в "конец" iterable. (5 доп баллов)
То есть при итерации по iter_append(iterable, item) мы сначала будем получать данные из iterable, а в самом конце одно значение item. Пример использования на рисунке Example3
Напишите этот генератор без явных циклов (без конструкций while, for и list comprehensions)
Не используйте никакие другие функции в теле iter_append
2. Сделайте функцию, которая "распаковывает" вложенные списки (5 доп баллов)
Уровень вложенности списков может быть любым. Пример использования на рисунке Example2.
Подсказка: используйте генераторы и синтаксис из предыдущего задания


Общие требования:
Наличие README с описанием и инструкцией по использованию поощрается
Сторонние библиотеки не используем
Форма сдачи - ссылка на пулл реквест в master/main ветку вашего репозитория BI_2021_Python
Решение оформить в виде Python скрипта

### Задание 1

In [34]:
path_to = r'sequences.fasta'

In [8]:
def gen_fasta(path:str):
    body = []
    stroka = str()
    with open(path, 'r') as file:
        body.append(file.readline().strip())
        for line in file:
            if line.startswith('>'):
                body.append(stroka)
                body.append(line.strip())
                stroka = str()
            else:
                stroka += line.strip()
    body.append(stroka)
    for value in body:
        yield value

#### Демонстрация

In [11]:
generator = gen_fasta(path_to)
print(generator)

<generator object gen_fasta at 0x00000211E4094D60>


In [12]:
for _ in range(4):
    print(next(generator))

>Seq1
MVELKEPFATLWRGKDPFEEVKTLQGEVFRELETRRTLRFEMAGKSYFLKWHRGTTLKEIIKNLLSLRMPVLGADREWNAIHRLRDVGVDTMYGVAFGEKGMNPLTRTSFIITEDLTPTISLEDYCADWATNPPDVRVKRMLIKRVATMVRDMHAAGINHRDCYICHFLLHLPFSGKEEELKISVIDLHRAQLRTRVPRRWRDKDLIGLYFSSMNIGLTQRDIWRFMKVYFAAPLKDILKQEQGLLSQAEAKATKIRERTIRKSL
>Seq2
MKLMLAEPFKSLWAGRDAFAEVEALKGEVYRELEARRTLRTEVDGRGFFVKIHRGIGWGEIFKNLFTAKLPVLGAGQEWKAIQRLQEVGVPTMTAVAYGEKGSNPADQHSFIVTEELAPTVSLEDFSIDWVKQPPQPALKRALIAEVARMTGMMHRAGVNHRDCYICHFLLHTDKPVTPEDFKLSVIDLHRAQTRPAITTRWRNKDLAALYFSALDIGLTRRDKLRFLKGYFQQPLRRILAEEASLLAWLEGKANKLYARKQRYGDAL


### Задание 2

In [3]:
import random

In [51]:
class GeneratorRex:
    """
    input: path to .fasta file
    replacement - replace one or more letters in the given sequence
    delit - remove a letter or piece in the given sequence
    inser - insert one or more letters into the given sequence
    chance - apply replacement, delit or inser with some probability to the passed sequence
    """
    def __init__(self, path_to_file):
        self.path_to_file = path_to_file
        self.box = []
        self.stroka = str()
        with open(self.path_to_file, 'r') as file:
            file.readline()
            for line in file:
                if line.startswith('>'):
                    self.box.append(self.stroka)
                    self.stroka = str()
                else:
                    self.stroka += line.strip()
        self.box.append(self.stroka)
        self._coursor = -1
        
    def replacement(self, line):
        lengh = len(line)
        line_list = list(line)
        amount_of_replace = range(int(lengh*round(random.choice([i*0.1 for i in range(5)]), 2)))
        indexes_for_replace = [random.randint(0, lengh-1) for _ in amount_of_replace]
        for i in indexes_for_replace:
            line_list[i] = random.choice(list(set(line)))  # replace to letter from line alphabet
        return ''.join(line_list)
    
    def delit(self, line):
        lengh = len(line)
        rand_chance = random.randint(0, 3)  # rand_chance == 3 probability 1/4
        if rand_chance == 3:  # big del
            boundary_list = sorted([random.randint(0, lengh) for _ in range(2)]) 
            return line[:boundary_list[0]] + line[boundary_list[1]:]
        else:  # short del
            boundary_num = random.randint(0, lengh) 
            return line[:boundary_num] + line[boundary_num+1:]
        
    def inser(self, line):
        lengh = len(line)
        rand_chance = random.randint(0, 3)  # rand_chance == 3 probability 1/4
        boundary = random.randint(0, lengh)
        if rand_chance == 3:
            insert_part = "".join([random.choice(list(set(line))) for _ in range(random.randint(0, int(lengh*0.5)))])
            return line[:boundary] + insert_part + line[boundary:]
        else:
            return line[:boundary] + random.choice(list(set(line))) + line[boundary:]
        
    def chance(self, line):
        coin = random.randint(0, 3)
        if coin >= 1:
            return self.replacement(line)
        else:
            coin = random.randint(0, 1)
            if coin:
                return self.inser(line)
            else:
                return self.delit(line)
    
    def __next__(self):    
        self._coursor += 1
        try:
            return self.chance(self.box[self._coursor])
        except IndexError:
            self._coursor = 0
            return self.chance(self.box[self._coursor])
       
    def __iter__(self):
        return self

#### Демонстрация класса

In [52]:
rex = GeneratorRex(path_to)

In [53]:
next(rex)

'MVELKGPFATLSRGKAPFVEVKTLQWEAERYLEHQHTLRFEMASKSNFLCWHRWTTSSEIIKNLLSLRMIVLGADRTWNYIHRLRDVGVDTMYCVAFGFKGMFPLTRTSYIITYDGTGVISLCDYEADKITNEPDVWVKRMLIKRVATWVCDTHAAGINTHCCYICFMLLHAYFSQMEEELKISVIDLHRAQLRTRVPRRWRDKDLIGLYFSWMKYGYTQRDIWWFDSVYFLAPLKDIEKQEQRHLSQAMAMATKRAERTTYKVL'

In [54]:
len(rex.box)

166

In [55]:
for i in range(168):
    intermediant = next(rex)
intermediant

'MRLVLEEPFKRLWNGRDPFEAVEALQGKVYRELEGRRTLRTEVDGRGYFVKIHRGIGWGEIAKNLLTAKLPVLGARQEWQAIRRLHEAGVATMTAVAYGERGSDPARQHSFIVTEELAPTVDLEVFSQDWRERPPPPRLKRALVEAVARMVGDMHRAGVNHRDCYICHFLLHTDKPVSADDFRLSVIDLHRAQTRDATPKRWRNKDLAALYFSALDGLTRRDKLRFLRTYFRRPLREILRDEAGLLAWMERQAEKLYERKQRYGDLL'